## Representation of the data on maps

### Package import

In [17]:
import geopandas as gpd
import folium
import numpy as np
import matplotlib.pyplot as plt
from shapely.geometry import LineString, Point
import osmnx as ox
import pandas as pd

### Open data extraction

In [ ]:
barrios=gpd.read_file("barrios/barris-barrios.shp")
puntos_carga = gpd.read_file("zonas-carrega-descarrega/zones-carrega-descarrega-zonas-carga-descarga.shp")

In [ ]:

# Asegúrate de que 'barrios' es un objeto de tipo GeoDataFrame y 'barrios.union_all()' sea un polígono válido
polygon = barrios.union_all()

# Descargamos el grafo dentro del polígono con los parámetros requeridos
graph = ox.graph_from_polygon(polygon, 
                              network_type="all",  # Tipo de red, elige el que desees ("walk", "drive", etc.)
                              simplify=False,  # No simplificar los nodos
                              retain_all=True,  # Mantener todos los componentes del grafo
                              truncate_by_edge=True)  # Mantener las calles que están parcialmente dentro y fuera del polígono

# Convertir el grafo a dos GeoDataFrames: uno para nodos y otro para aristas
gdf_nodes, gdf_edges = ox.graph_to_gdfs(graph, nodes=True, edges=True)

# Filtrar gdf_edges eliminando las filas con esos valores en la columna 'access'
valores_no_utiles_access = ['private', 'no', 'customers']
gdf_edges_filtrado = gdf_edges[~gdf_edges['access'].isin(valores_no_utiles_access)]

# Filtrar las filas para eliminar los tipos de calle no deseados
valores_no_utiles_highway = ['elevator', 'busway', 'corridor', 'unclassified', 'services', 'cycleway', 'steps']
gdf_edges_filtrado = gdf_edges_filtrado[~gdf_edges_filtrado['highway'].isin(valores_no_utiles_highway)]

# Obtener los nodos asociados a las aristas filtradas, utilizando el MultiIndex
nodos_filtrados = pd.concat([gdf_edges_filtrado.index.get_level_values('u').to_series(), 
                             gdf_edges_filtrado.index.get_level_values('v').to_series()]).unique()

# Filtrar gdf_nodes para que solo contenga los nodos asociados a las aristas restantes
gdf_nodes_filtrado = gdf_nodes[gdf_nodes.index.isin(nodos_filtrados)]

# Asegurarse de no eliminar nodos que aún estén conectados a aristas válidas
# Obtener los nodos asociados a las aristas eliminadas para evitar que esos nodos sean filtrados
nodos_no_utilizados = set(gdf_edges.index.get_level_values('u')).union(set(gdf_edges.index.get_level_values('v'))) - set(nodos_filtrados)

# Asegurarse de que los nodos no eliminados no se pierdan
gdf_nodes_filtrado = gdf_nodes_filtrado[~gdf_nodes_filtrado.index.isin(nodos_no_utilizados)]

# Eliminar las aristas que tienen nodos que han sido eliminados (es decir, nodos que no están en gdf_nodes_filtrado)
gdf_edges_filtrado = gdf_edges_filtrado[gdf_edges_filtrado.index.get_level_values('u').isin(gdf_nodes_filtrado.index) &
                                         gdf_edges_filtrado.index.get_level_values('v').isin(gdf_nodes_filtrado.index)]
                                        
# Definir los valores que indican que no hay restricción
sin_restriccion = {'motorway', 'residential', 'secondary', 'living_street', 
                   'primary_link', 'primary', 'tertiary', 'trunk',
                   'service', 'tertiary_link', 'trunk_link', 'secondary_link',
                   'motorway_link', 'road'}

# Crear la nueva columna 'Model3' con True o False
gdf_edges_filtrado["Model3"] = ~gdf_edges_filtrado["highway"].isin(sin_restriccion)                

# Guardar
gdf_edges_filtrado.to_file("openstreet/open-valencia-edges.shp")
gdf_nodes_filtrado.to_file("openstreet/open-valencia-nodes.shp")

'\n# Asegúrate de que \'barrios\' es un objeto de tipo GeoDataFrame y \'barrios.union_all()\' sea un polígono válido\npolygon = barrios.union_all()\n\n# Descargamos el grafo dentro del polígono con los parámetros requeridos\ngraph = ox.graph_from_polygon(polygon, \n                              network_type="all",  # Tipo de red, elige el que desees ("walk", "drive", etc.)\n                              simplify=False,  # No simplificar los nodos\n                              retain_all=True,  # Mantener todos los componentes del grafo\n                              truncate_by_edge=True)  # Mantener las calles que están parcialmente dentro y fuera del polígono\n\n# Convertir el grafo a dos GeoDataFrames: uno para nodos y otro para aristas\ngdf_nodes, gdf_edges = ox.graph_to_gdfs(graph, nodes=True, edges=True)\n\n# Filtrar gdf_edges eliminando las filas con esos valores en la columna \'access\'\nvalores_no_utiles_access = [\'private\', \'no\', \'customers\']\ngdf_edges_filtrado = gdf_